# TMP91FW27UG Bootloader SPA

In this lab, we'll be looking at the [TMP91FW27UG](https://toshiba.semicon-storage.com/us/semiconductor/product/microcontrollers/detail.TMP91FW27UG.html) device. You can find a link to the [datasheet PDF here](https://toshiba.semicon-storage.com/info/docget.jsp?did=10173&prodName=TMP91FW27UG), or there is a local copy on the USB stick.

Our objective is to bypass the security mechanism of this device, so take a look at the datasheet and try to udnerstand what type of code protection the device has. When researcing this, it can help to search for keywords like "security", "fuse", "password", "readback", "code security", "read out", or "bootloader". Different manufactures use different technologies or words.

Try to understand the following before continuing:

1. What built-in bootloader or programming method is included in the device?
2. Can you access the ROM for the bootloader? How much detail is there in the datasheet? Is there anything they might be lying about?
3. What access control level is part of the protection mechanism?
4. What pin(s) do we need to control for the bootloader?

## Talking to the Bootloader

In order to talk to the bootloader, we included a TMP91FW27UG target board. This board includes the TMP91FW27UG microcontroller. You can see the schematic and design for this board [here](https://github.com/newaetech/chipwhisperer-target-cw308t/tree/main/CW312T_TMP91FW) if you wish.

The important points about the pinout are:

1. The clock is driven from the ChipWhisperer, using the normal "clkgen" output.
2. The serial is connected as `tio1 = "serial_tx"`, `tio2 = "serial_rx"`
3. The BOOT pin is `pdic`, set it low (`False` or `0`) for bootloader, high (`True` or `1`) for normal boot

The following small snippets may be helpful: one shows how to calculate the checksum required, the other sends a command and returns the response. The response is returned in two formats (sometimes you'll want one or the other).

You may need to modify these commands further in your own code.

In [ ]:
def calc_checksum(r):
    s = 0
    for c in r:
        s += c
    cs = 0x100 - (s & 0xff)
    return cs

def tx_rx(cmd, expectedlen=0, rxlen=100):
    target.write(cmd)
    response = target.read(rxlen)
    if expectedlen:
        if len(response) != expectedlen:
            raise IOError("Unexpected response length %d (data: %s)"%(len(response), str(response)))

    responsehex = [hex(ord(c)) for c in response]
    return response, responsehex

## Selecting a Command

There are several bootloader commands to choose from, as shown in the table here:

| Name                     | Command | Needs Password? | Protection Bit Blocked? |
|--------------------------|---------|-----------------|-------------------------|
| RAM Transfer             | 0x10    | ?               | ?                       |
| Flash memory SUM         | 0x20    | No              | ?                       |
| Product Information Read | 0x30    | ?               | ?                       |
| Chip Erase               | 0x40    | No              | No                      |
| Protection Bit Set       | 0x60    | ?               | ?                       |

Try to fill in the rest of the "Needs Password" and "Protection Bit Blocked" columns. It will help you understand the most useful command for doing an SPA attack.

**HINT: You want a command which requires the password but isn't blocked by the protection bit. There is one suitable command only in the particular bootloader. If we didn't have such a command, we'd need to perform a fault injection attack to bypass the protection bit.**

Once you have a command, you can see the general flow of the command structure from the datasheet. You'll need some code to do the following:

1. Set the BOOT flag as required.
2. Reset the device.
3. Send the required command, check the response.

To write to the device, you'll need to send hex data like e.g.:

```python
target.write([0x86])
```

When you read data, you can specify how many bytes to read. Remember there might be garbage in the serial line, so you might need to clear the output by trying to read too much data first:

```python
target.read(1000)
```

Or use the flush command:

```python
target.flush()
```

So a typical code-flow should look like:

```python
import time
scope.io.pdic = ?
scope.io.nrst = False
time.sleep(0.1)
scope.io.nrst = True
time.sleep(0.1)
target.flush() # This flushes old data out
target.write([0x86])
data = target.read(1)
print(data)
```

You might notice the `data` is just a string. You can convert it to a list with different commands. Try converting to `bytes` or `bytearray`. You can force this to just print as a list with:

```python
[ord(c) for c in data]
```

The `ord(c)` function converts an ascii char into an integer. You can also convert this to  hex by wrapping in the `hex()` or a formatting call, such as:

```python
['%02x'%ord(c) for c in data]
```

See the following examples of the conversion of a single character:

In [ ]:
ord('a')

In [ ]:
'%02x'%ord('a')

Try adding your code here to talk to the device. We've given you the required setup code to start with:

In [ ]:
PLATFORM = ""
%run ../Setup_Scripts/Setup_Generic.ipynb

scope.clock.clkgen_freq = 25E6 #25 MHz on examples
target.baud = 9600 #9600 baud rate
scope.io.tio1 = "serial_tx"
scope.io.tio2 = "serial_rx"

## Triggering & Plotting SPA Attempt

Next, we need to do an SPA attempt. To do this, we're going to trigger on `tio1`, which is the serial line. This will work because we simply trigger on the serial data we are sending to the device, which is the first byte of the password.

We can look at more complex triggering methods later on.

In [ ]:
scope.trigger.triggers = "tio1"
scope.clock.adc_mul = 1
scope.adc.offset = 0
scope.adc.samples = 100000
scope.adc.presamples = 0

In [ ]:
#raise NotImplementedError ("Add your Code here instead, delete this")
scope.io.pdic = 0
scope.io.nrst = False
time.sleep(0.1)
scope.io.nrst = True
time.sleep(0.1)
# You need to add the code here to set PDIC to the right value & reset it from above.

In [ ]:
scope.arm()
target.write([0x86])
scope.capture()
trace = scope.get_last_trace()
cw.plot(trace)

What you've done above is just trigger on the initial sync packet (assuming you left the code as-is). In which case you'll get something like this:

![](img/tmp91boot.png)

What you need to do now is use that to trigger on the **password** being sent. To start with, make a `reset_target()` function to simplify the loop which resets the device each time:

In [ ]:
def reset_target():
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = True
    time.sleep(0.05)

Then try making a small loop to test some different first characters of the password. Here's an example starting point for the loop.

Modify it to correctly setup the function you identified above, and then plot the actual password comparison. At this point things won't syncronize well, so we'll work on that afterwards.

**HINT: You don't actually need to send the full command - sending only up to the first byte of the password will work**

These methods should be helpful for this part:

In [ ]:
tx_rx??

In [ ]:
calc_checksum??

In [ ]:
import numpy as np
from tqdm.notebook import trange, tqdm

plot = cw.plot()
scope.trigger.module = 'basic'

for i in range(20, 23):
    reset_target()
    target.flush()
    
    raise NotImplementedError("Requires your code here!")
    
    #scope.UARTTrigger.set_pattern_match(0, chr(i))
    scope.arm()
    target.write(chr(i))
    scope.capture()
    
    trace = scope.get_last_trace()
      
    plot *= cw.plot(trace).opts(alpha=0.25)
display(plot)

Things will look unsyncronized, which is a problem. For example, when running it my test looked like this:

![](img/tmp91unsync.png)

## Resync with Software

One of the problems above is you need to resyncronize the power traces. We'll do this with a simple software routine here. Doing this will require us to:

1. Manually decide on what looks like an interesting point. Luckily that is easy in this example.
2. Pass the interesting points to the following function, along with traces we want to resyncronize.
3. Plot the resyncronized traces.


In [ ]:
def resync_sad(trace, ref, ref_range, pad=True, copy=True):
    _, offset = sad(trace, ref, ref_range)
    
    if copy:
        inlen = len(trace)
        output = np.zeros(inlen)
        if offset < 0:
            output[-offset:] = trace[:(inlen + offset)]
        else:
            output[:(inlen - offset)] = trace[offset:]
    else:
        raise NotImplementedError("Insert your code")
    
    return output    

def sad(full_trace, pattern, pattern_range=None, valid_only=True):
    
    tlen = len(full_trace)
    
    psum = np.zeros(tlen)
    temp = np.zeros(tlen)
    
    if pattern_range:
        plen = pattern_range[1] - pattern_range[0]
        pstart = pattern_range[0]
    else:
        plen = len(pattern)
        pstart = 0
        

    for j in range(plen-1, -1, -1):
        #NB - due to way we index through array, don't need to reset this back to zero
        #temp = np.zeros(100000)
        np.subtract(full_trace[j:], pattern[pstart+j], out=temp[0:(100000-j)])
        psum += abs(temp)

    if valid_only:
        psum = psum[0:-plen]
    
    offset = np.argmin(psum)    
    offset = np.argmin(psum) - pattern_range[0] + (pattern_range[1] - pattern_range[0])
    
    return psum, offset

In [ ]:
cw.plot(trace)

In [ ]:
start = 19300 #ADJUST THIS
end = 19680 #ADJUST THIS
trace1 = trace
cw.plot(trace1[start:end])

Adjust the above start & end parameters until you get a nice-looking `trace1`. For example, something like this should be in your view:

![](img/tmp91resync1.png)

Finally, run code like the following and see if it resyncronized all the traces:

In [ ]:
import numpy as np
from tqdm.notebook import trange, tqdm

go = True

i = 0

diffs = []
traces = []

plot = cw.plot()

for i in range(20, 24):
    reset_target()
    target.flush()
    
    raise NotImplementedError("Requires your code here!")
    
    scope.arm()
    target.write(chr(i))
    scope.capture()
    
    trace = scope.get_last_trace()
    #NOTE - you'll need to adjust the -300 number currently to target the center of the actual work

    trace = resync_sad(trace, trace1, (start,end))
    
    trace_excerpt = trace[start-300:end-300]
    
    diff_trace = trace_excerpt - trace1[start:end]
    traces.append(trace_excerpt)
    
    diff = np.sum(abs(diff_trace)[80:180])
    diffs.append(diff)
    print("%x %f"%(i, diff))
    
    plot *= cw.plot(trace_excerpt).opts(alpha=0.25)
display(plot)

## SPA Attempt with Software

With the syncronization working, you now need to script the full attack. You'll notice in the previous example it started to print the deltas between `trace1` and the guess - this might be a good starting point for the full attack!

Unlike previous labs, this one is more open ended. It'll be up to you to do most of this on your own. If you do get stuck, you can steal some of the code from the following notebook: https://github.com/colinoflynn/samsung-ovens-deconstructed/blob/master/notebooks/Oven%20TMP91%20Hacky%20Test.ipynb

## UART Trigger

The Husky also includes a module that allows us to trigger on UART Data. If you've got extra time, you may want to try this trigger module out to see how it affects trace synchronization. Check out the documentation for this module at https://chipwhisperer.readthedocs.io/en/latest/scope-api.html#scope-uarttrigger

**HINT: `scope.adc.presamples` (https://chipwhisperer.readthedocs.io/en/latest/scope-api.html#chipwhisperer.capture.scopes._OpenADCInterface.TriggerSettings.presamples) will also be useful here**